In [80]:
"""TVAE module."""

import numpy as np
import pandas as pd
import torch
from torch.nn import Linear, Module, Parameter, ReLU, Sequential, GELU
from torch.nn.functional import cross_entropy
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

from ctgan.data_transformer import DataTransformer
from ctgan.synthesizers.base import BaseSynthesizer, random_state
from sdv.metadata import SingleTableMetadata

class Encoder(Module):
    """Encoder for the modified TVAE based on SiloFuse architecture."""
    def __init__(self, data_dim, embedding_dim=32, hidden_dim=1024):  # Embedding and hidden dim set to 32 and 1024 respectively.
        super(Encoder, self).__init__()
        print("Dimensions", data_dim, embedding_dim, hidden_dim)
        # Three linear layers with GELU activation
        self.seq = Sequential(
            Linear(data_dim, hidden_dim),
            GELU(),
            Linear(hidden_dim, hidden_dim),
            GELU(),
            Linear(hidden_dim, embedding_dim),
            GELU()  # GELU activation for the last layer as well
        )

        # Latent mean and log variance (still assuming Gaussian latent space)
        self.fc1 = Linear(hidden_dim, embedding_dim)
        self.fc2 = Linear(hidden_dim, embedding_dim)

    def forward(self, input_):
        """Encode the passed `input_`."""
        feature = self.seq(input_)
        mu = self.fc1(feature)
        logvar = self.fc2(feature)
        std = torch.exp(0.5 * logvar)
        return mu, std, logvar


class Decoder(Module):
    """Decoder for the modified TVAE based on SiloFuse architecture."""

    def __init__(self, embedding_dim=32, hidden_dim=1024, data_dim=None):
        super(Decoder, self).__init__()
        
        # Three linear layers with GELU activation
        self.seq = Sequential(
            Linear(embedding_dim, hidden_dim),
            GELU(),
            Linear(hidden_dim, hidden_dim),
            GELU(),
            Linear(hidden_dim, data_dim)  # Output layer with no activation as it is handled later
        )
        self.sigma = Parameter(torch.ones(data_dim) * 0.1)

    def forward(self, input_):
        """Decode the passed `input_`."""
        return self.seq(input_), self.sigma


def _loss_function(recon_x, x, sigmas, mu, logvar, output_info, factor):
    st = 0
    loss = []
    for column_info in output_info:
        for span_info in column_info:
            if span_info.activation_fn != 'softmax':
                ed = st + span_info.dim
                std = sigmas[st]
                eq = x[:, st] - torch.tanh(recon_x[:, st])
                loss.append((eq**2 / 2 / (std**2)).sum())
                loss.append(torch.log(std) * x.size()[0])
                st = ed

            else:
                ed = st + span_info.dim
                loss.append(
                    cross_entropy(
                        recon_x[:, st:ed], torch.argmax(x[:, st:ed], dim=-1), reduction='sum'
                    )
                )
                st = ed

    assert st == recon_x.size()[1]
    KLD = -0.5 * torch.sum(1 + logvar - mu**2 - logvar.exp())
    return sum(loss) * factor / x.size()[0], KLD / x.size()[0]


class TVAE(BaseSynthesizer):
    """TVAE."""

    def __init__(
        self,
        embedding_dim=128,
        compress_dims=128, #(128, 128),
        decompress_dims=128, #(128, 128),
        l2scale=1e-5,
        batch_size=500,
        epochs=300,
        loss_factor=2,
        cuda=False,
        verbose=True,
    ):
        self.embedding_dim = embedding_dim
        self.compress_dims = compress_dims
        self.decompress_dims = decompress_dims

        self.l2scale = l2scale
        self.batch_size = batch_size
        self.loss_factor = loss_factor
        self.epochs = epochs
        self.loss_values = pd.DataFrame(columns=['Epoch', 'Batch', 'Loss'])
        self.verbose = verbose
        self.encoder = None

        if not cuda or not torch.cuda.is_available():
            device = 'cpu'
        elif isinstance(cuda, str):
            device = cuda
        else:
            device = 'cuda'

        self._device = torch.device(device)

    @random_state
    def fit(self, train_data, discrete_columns=()):
        """Fit the TVAE Synthesizer models to the training data.

        Args:
            train_data (numpy.ndarray or pandas.DataFrame):
                Training Data. It must be a 2-dimensional numpy array or a pandas.DataFrame.
            discrete_columns (list-like):
                List of discrete columns to be used to generate the Conditional
                Vector. If ``train_data`` is a Numpy array, this list should
                contain the integer indices of the columns. Otherwise, if it is
                a ``pandas.DataFrame``, this list should contain the column names.
        """
        self.transformer = DataTransformer()
        self.transformer.fit(train_data, discrete_columns)
        train_data = self.transformer.transform(train_data)
        dataset = TensorDataset(torch.from_numpy(train_data.astype('float32')).to(self._device))
        loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True, drop_last=False)

        data_dim = self.transformer.output_dimensions
        print(data_dim, self.compress_dims, self.embedding_dim)
        self.encoder = Encoder(data_dim, self.compress_dims, self.embedding_dim) #.to(self._device)
        self.decoder = Decoder(self.embedding_dim, self.decompress_dims, data_dim) #.to(self._device)
        optimizerAE = Adam(
            list(self.encoder.parameters()) + list(self.decoder.parameters()), weight_decay=self.l2scale
        )

        self.loss_values = pd.DataFrame(columns=['Epoch', 'Batch', 'Loss'])
        iterator = tqdm(range(self.epochs), disable=(not self.verbose))
        if self.verbose:
            iterator_description = 'Loss: {loss:.3f}'
            iterator.set_description(iterator_description.format(loss=0))

        for i in iterator:
            loss_values = []
            batch = []
            for id_, data in enumerate(loader):
                optimizerAE.zero_grad()
                real = data[0].to(self._device)
                mu, std, logvar = self.encoder(real)
                eps = torch.randn_like(std)
                emb = eps * std + mu
                rec, sigmas = self.decoder(emb)
                loss_1, loss_2 = _loss_function(
                    rec,
                    real,
                    sigmas,
                    mu,
                    logvar,
                    self.transformer.output_info_list,
                    self.loss_factor,
                )
                loss = loss_1 + loss_2
                loss.backward()
                optimizerAE.step()
                self.decoder.sigma.data.clamp_(0.01, 1.0)

                batch.append(id_)
                loss_values.append(loss.detach().cpu().item())

            epoch_loss_df = pd.DataFrame({
                'Epoch': [i] * len(batch),
                'Batch': batch,
                'Loss': loss_values,
            })
            if not self.loss_values.empty:
                self.loss_values = pd.concat([self.loss_values, epoch_loss_df]).reset_index(
                    drop=True
                )
            else:
                self.loss_values = epoch_loss_df

            if self.verbose:
                iterator.set_description(
                    iterator_description.format(loss=loss.detach().cpu().item())
                )

    @random_state
    def sample(self, samples):
        """Sample data similar to the training data.

        Args:
            samples (int):
                Number of rows to sample.

        Returns:
            numpy.ndarray or pandas.DataFrame
        """
        self.decoder.eval()

        steps = samples // self.batch_size + 1
        data = []
        for _ in range(steps):
            mean = torch.zeros(self.batch_size, self.embedding_dim)
            std = mean + 1
            noise = torch.normal(mean=mean, std=std).to(self._device)
            fake, sigmas = self.decoder(noise)
            fake = torch.tanh(fake)
            data.append(fake.detach().cpu().numpy())

        data = np.concatenate(data, axis=0)
        data = data[:samples]
        return self.transformer.inverse_transform(data, sigmas.detach().cpu().numpy())

    def generate_latents(self, data):
        """Generate latent vectors from the input data using the trained encoder.

        Args:
            data (numpy.ndarray or pandas.DataFrame):
                Input data to encode. Must match the format used for training.

        Returns:
            numpy.ndarray: Latent vectors generated by the encoder.
        """
        self.encoder.eval()
        data = self.transformer.transform(data)
        data_tensor = torch.from_numpy(data.astype('float32')).to(self._device)
        with torch.no_grad():
            mu, _, _ = self.encoder(data_tensor)  # Use mu as the latent vector
        return mu.cpu().numpy()

    def set_device(self, device):
        """Set the `device` to be used ('GPU' or 'CPU)."""
        self._device = device
        self.decoder.to(self._device)

In [81]:
def categorical_column_indices(metadata_dict):
    categorical_indices = []
    columns = metadata_dict.get('columns', {})
    column_names = list(columns.keys())[:-1]  # Exclude the last key
    for index, column_name in enumerate(column_names):
        column_data = columns[column_name]
        if column_data.get('sdtype') == 'categorical':
            categorical_indices.append(index)
    return categorical_indices

In [84]:
def generate_and_save_latent(model, source="../data/raw/bank.csv", path="../data/processed/bank,.csv"):
    DATA_PATH = source
    df = pd.read_csv(DATA_PATH, sep=",")
    actual_data = df.iloc[:, :-1]
    outcomes = df.iloc[:, -1]

    latents = []
    metadata = SingleTableMetadata()
    meta = metadata.detect_from_csv(source)

    discrete_columns = categorical_column_indices(metadata.to_dict())
    print(discrete_columns)
    model.fit(actual_data, discrete_columns)
    latents = model.generate_latents(actual_data)
    # unbatched_latent = torch.cat(latents, dim=0)

    latents_df = pd.DataFrame(latents) #(unbatched_latent)
    outcomes_df = pd.DataFrame(outcomes)
    # Save DataFrame to a CSV file
    data_with_outcomes = pd.concat([latents_df, outcomes_df], axis=1)

    data_with_outcomes.to_csv(path, index=False)

In [85]:
model = TVAE()
generate_and_save_latent(model)

[5, 7, 9, 10, 11, 12]
90 128 128
Dimensions 90 128 128


Loss: -53.439: 100%|██████████| 300/300 [31:30<00:00,  6.30s/it]   
